In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, K = 19

In [3]:
a = 0.25
num_params=19
num_sims = 100

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.51 & 1.00 & 0.95 & 0.96 & 0.93 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.01 & 0.02 & 0.01   \\
Model 2 & 0.49 & 0.00 & 0.04 & 0.03 & 0.05 & 0.03   \\
\hline
\end{tabular}
(array([0.51, 0.  , 0.49]), array([1., 0., 0.]), array([0.95, 0.01, 0.04]), array([0.96, 0.01, 0.03]), array([0.93, 0.02, 0.05]), array([0.96, 0.01, 0.03]), -10.033539074645867, 4.9930544532813315, 5.152304289202481)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.76 & 0.85 & 0.96 & 0.96 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.02 & 0.01 & 0.01   \\
Model 2 & 0.24 & 0.15 & 0.02 & 0.02 & 0.02 & 0.01   \\
\hline
\end{tabular}
(array([0.76, 0.  , 0.24]), array([0.85, 0.  , 0.15]), array([0.96, 0.02, 0.02]), array([0.96, 0.02, 0.02]), array([0.97, 0.01, 0.02]), array([0.98, 0.01, 0.01]), -8.85948456970108, 5.755323989757422, 6.536350571731881)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.85 & 0.85 & 0.95 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.04 & 0.02 & 0.02 & 0.02   \\
Model 2 & 0.15 & 0.15 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.85, 0.  , 0.15]), array([0.85, 0.  , 0.15]), array([0.95, 0.04, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), -7.582837491074562, 8.015199258633569, 8.508745660088305)


In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.95 & 0.95 & 0.95 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.04 & 0.03 & 0.03 & 0.03   \\
Model 2 & 0.07 & 0.07 & 0.01 & 0.02 & 0.02 & 0.01   \\
\hline
\end{tabular}
(array([0.93, 0.  , 0.07]), array([0.93, 0.  , 0.07]), array([0.95, 0.04, 0.01]), array([0.95, 0.03, 0.02]), array([0.95, 0.03, 0.02]), array([0.96, 0.03, 0.01]), -6.615029282871518, 10.374950433834373, 11.255924000845704)


# a = 0

In [8]:
a = 0.0
num_params=19
num_sims = 100

In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.47 & 1.00 & 0.99 & 0.99 & 0.99 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.53 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.47, 0.  , 0.53]), array([1., 0., 0.]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), -8.944468002780134, 3.142047674270612, 4.278751634463372)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.48 & 0.98 & 0.99 & 0.99 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.52 & 0.02 & 0.01 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.48, 0.  , 0.52]), array([0.98, 0.  , 0.02]), array([0.99, 0.  , 0.01]), array([0.99, 0.  , 0.01]), array([1., 0., 0.]), array([1., 0., 0.]), -8.916438539774944, 2.716938058637645, 4.313741392604251)


# a  = .25, k= 4, something not right?

In [11]:
a = 0.25
num_params=4
num_sims = 100

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.96 & 0.95 & 0.96 & 0.97   \\
Model 1 & 0.02 & 0.02 & 0.03 & 0.04 & 0.03 & 0.02   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.96, 0.03, 0.01]), array([0.95, 0.04, 0.01]), array([0.96, 0.03, 0.01]), array([0.97, 0.02, 0.01]), 0.22923351523960028, 5.003353863188334, 5.200168299428068)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.89 & 0.89 & 0.89 & 0.88 & 0.91   \\
Model 1 & 0.10 & 0.10 & 0.10 & 0.11 & 0.12 & 0.09   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.89, 0.1 , 0.01]), array([0.89, 0.1 , 0.01]), array([0.89, 0.1 , 0.01]), array([0.89, 0.11, 0.  ]), array([0.88, 0.12, 0.  ]), array([0.91, 0.09, 0.  ]), 3.583040911159651, 7.6239791705516815, 7.201621719641817)


# a  = .25, k= 9

In [14]:
a = 0.25
num_params=9
num_sims = 100

In [15]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.96 & 0.97 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.03 & 0.03 & 0.02   \\
Model 2 & 0.04 & 0.04 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.  , 0.04]), array([0.96, 0.  , 0.04]), array([0.96, 0.03, 0.01]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.98, 0.02, 0.  ]), -3.0014088091424127, 4.901793385197222, 5.663682383134375)


In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.96 & 0.96 & 0.96 & 0.97   \\
Model 1 & 0.01 & 0.01 & 0.03 & 0.03 & 0.03 & 0.02   \\
Model 2 & 0.02 & 0.02 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([0.96, 0.03, 0.01]), array([0.96, 0.03, 0.01]), array([0.96, 0.03, 0.01]), array([0.97, 0.02, 0.01]), -1.6281890068906366, 7.150834880047641, 7.7742604351754325)


In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.97 & 0.97 & 0.97 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.03 & 0.03 & 0.02   \\
Model 2 & 0.11 & 0.03 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.89, 0.  , 0.11]), array([0.97, 0.  , 0.03]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.98, 0.02, 0.  ]), -2.6993748414892114, 4.095079382572336, 4.136933292524103)


# a  = 0, k= 9

In [18]:
a = 0.00 
num_params=9
num_sims = 100

In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.98 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.05 & 0.02 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.  , 0.05]), array([0.98, 0.  , 0.02]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -3.422932398346382, 1.8465998107207717, 2.7620361348269613)


In [20]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.99 & 0.98 & 0.98 & 0.97 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.02 & 0.03 & 0.01   \\
Model 2 & 0.05 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.  , 0.05]), array([0.99, 0.  , 0.01]), array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.99, 0.01, 0.  ]), -3.686723496381256, 2.1141918546759215, 2.971956857913732)


# evidence of power

In [21]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [22]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.69 & 0.69 & 0.38 & 0.38 & 0.40 & 0.42   \\
Model 1 & 0.31 & 0.31 & 0.62 & 0.62 & 0.60 & 0.58   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.69, 0.31, 0.  ]), array([0.69, 0.31, 0.  ]), array([0.38, 0.62, 0.  ]), array([0.38, 0.62, 0.  ]), array([0.4, 0.6, 0. ]), array([0.42, 0.58, 0.  ]), 7.573959865331092, 4.948455202684164, 5.180933225406859)


In [23]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.78 & 0.78 & 0.42 & 0.39 & 0.40 & 0.48   \\
Model 1 & 0.22 & 0.22 & 0.58 & 0.61 & 0.60 & 0.52   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.78, 0.22, 0.  ]), array([0.78, 0.22, 0.  ]), array([0.42, 0.58, 0.  ]), array([0.39, 0.61, 0.  ]), array([0.4, 0.6, 0. ]), array([0.48, 0.52, 0.  ]), 7.688183288168144, 4.8914582728115334, 5.384313850010068)


In [24]:
a1,a2 = 0.00,np.sqrt(1.09-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.08 & 0.22 & 0.54 & 0.55 & 0.52 & 0.62   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.92 & 0.78 & 0.46 & 0.45 & 0.48 & 0.38   \\
\hline
\end{tabular}
(array([0.08, 0.  , 0.92]), array([0.22, 0.  , 0.78]), array([0.54, 0.  , 0.46]), array([0.55, 0.  , 0.45]), array([0.52, 0.  , 0.48]), array([0.62, 0.  , 0.38]), -14.021331022080094, 5.261185634682602, 5.138907212734967)


In [25]:
a1,a2 =  0.00,np.sqrt(1.09**.5-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.15 & 0.22 & 0.57 & 0.57 & 0.59 & 0.65   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.85 & 0.78 & 0.43 & 0.43 & 0.41 & 0.35   \\
\hline
\end{tabular}
(array([0.15, 0.  , 0.85]), array([0.22, 0.  , 0.78]), array([0.57, 0.  , 0.43]), array([0.57, 0.  , 0.43]), array([0.59, 0.  , 0.41]), array([0.65, 0.  , 0.35]), -13.231390618477926, 4.928848448149421, 5.175238744957291)
